Based on [this tutorial](https://github.com/huggingface/notebooks/blob/master/transformers_doc/training.ipynb)

In [ ]:
import pandas as pd

from utils import load_dataset
mosi = load_dataset()

In [ ]:
import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")
# tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")
model = transformers.AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
# model = transformers.AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).cuda()

In [3]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_mosi = mosi.map(tokenize_function, batched=True)

In [6]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments("test_trainer", per_device_train_batch_size=2)
trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_mosi["train"], eval_dataset=tokenized_mosi["val"]
)

In [7]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 1283
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 963
/home/dstratton/miniconda3/envs/InterpretableMultimodal/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
/home/dstratton/miniconda3/envs/InterpretableMultimodal/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=963, training_loss=0.4568028910509151, metrics={'train_runtime': 180.2524, 'train_samples_per_second': 21.353, 'train_steps_per_second': 5.343, 'total_flos': 1012714452080640.0, 'train_loss': 0.4568028910509151, 'epoch': 3.0})

In [8]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_mosi["train"],
    eval_dataset=tokenized_mosi["val"],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 229
  Batch size = 16


{'eval_loss': 0.8881937265396118,
 'eval_accuracy': 0.8165938864628821,
 'eval_runtime': 2.5546,
 'eval_samples_per_second': 89.644,
 'eval_steps_per_second': 5.872}

In [20]:
predictions = trainer.predict(tokenized_mosi["test"])

# test_accuracy is correct, predictions for each class, you can take argmax
predictions.metrics

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 686
  Batch size = 16
/home/dstratton/miniconda3/envs/InterpretableMultimodal/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'test_loss': 1.0470805168151855,
 'test_accuracy': 0.7871720116618076,
 'test_runtime': 7.3155,
 'test_samples_per_second': 93.773,
 'test_steps_per_second': 5.878}